In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import math
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.statespace.varmax import VARMAX

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error 



In [2]:
data = pd.read_csv('C:\\Users\\user\\OneDrive\\Desktop\\Research Project\\Codes\\final_sentiment_data.csv')

In [17]:
data.keys()

Index(['datetime', 'sentiment_vader', 'positive_count', 'neutral_count',
       'negative_count', 'ratio_positive', 'sentiment_polarization',
       'net_sentiment', 'positive_to_total', 'negative_to_total', 'Close',
       'Volume BTC', 'Volume USDT', 'trade_count'],
      dtype='object')

## Process the data for machine learing model training

In [3]:
data.set_index('datetime', inplace=True)

In [4]:
data = data.loc['2021-01-01 00:00:00':'2022-05-31 00:00:00']

In [5]:
timelag = 2
for i in range(1, timelag+1):
    data[f'Close_{i}hr'] = data['sentiment_polarization'].shift(i)
    data[f'Close_lag_back_{i}hr'] = data['sentiment_polarization'].shift(-i)
    data[f'sentiment_lag_{i}hr'] = data['sentiment_polarization'].shift(i)
    data[f'sentiment_lag_back_{i}hr'] = data['sentiment_polarization'].shift(-i)
    data[f'trade_count_{i}hr'] = data['trade_count'].shift(i)
    data[f'trade_count_back_{i}hr'] = data['trade_count'].shift(-i)
    data[f'Volume USDT_{i}hr'] = data['trade_count'].shift(i)
    data[f'Volume USDT_back_{i}hr'] = data['trade_count'].shift(-i)
    data[f'Volume BTC_{i}hr'] = data['trade_count'].shift(i)
    data[f'Volume BTC_back_{i}hr'] = data['trade_count'].shift(-i)


In [21]:
data.shape

(12311, 33)

In [22]:
data.keys

<bound method NDFrame.keys of                      sentiment_vader  positive_count  neutral_count  \
datetime                                                              
2021-01-01 00:00:00         0.583991        0.051003       0.009387   
2021-01-01 01:00:00         0.439338        0.089818       0.023956   
2021-01-01 02:00:00         0.431446        0.082880       0.028589   
2021-01-01 03:00:00         0.438948        0.063191       0.023286   
2021-01-01 04:00:00         0.537289        0.063004       0.015788   
...                              ...             ...            ...   
2022-05-30 20:00:00         0.367085        0.106319       0.039073   
2022-05-30 21:00:00         0.357056        0.136134       0.051387   
2022-05-30 22:00:00         0.375190        0.125258       0.047607   
2022-05-30 23:00:00         0.389964        0.098631       0.033526   
2022-05-31 00:00:00         0.414391        0.097131       0.027979   

                     negative_count  ratio_pos

In [6]:
null_count = data.isnull().sum().sum()

# Print the result
print("Number of null values:", null_count)

Number of null values: 30


In [6]:
data.dropna(inplace=True)
data.drop(['ratio_positive', 'positive_count', 'neutral_count',	'negative_count', 'sentiment_vader', 'positive_to_total',	'net_sentiment', 'positive_to_total', 'negative_to_total'], axis = 1, inplace= True)

In [7]:
# Assume df is your DataFrame and 'Close' is your target column
target_column = 'Close'

# Compute the correlation of each numeric column with the target column
correlations = data.apply(lambda x: x.corr(data[target_column]))

# Drop the target column from the correlations
correlations = correlations.drop(target_column)

# Sort the correlation values in descending order
sorted_columns = correlations.sort_values(ascending=False).index

# Reorder the DataFrame columns based on the sorted correlation values
sorted_df = data[[target_column] + list(sorted_columns)]

In [27]:
sorted_df

,Close,sentiment_polarization,Close_1hr,sentiment_lag_1hr,Close_2hr,sentiment_lag_2hr,Close_lag_back_1hr,sentiment_lag_back_1hr,sentiment_lag_back_2hr,Close_lag_back_2hr,...,Volume USDT_back_1hr,trade_count_back_1hr,trade_count_2hr,Volume USDT_2hr,Volume BTC_2hr,Volume USDT_1hr,trade_count_1hr,Volume BTC_1hr,trade_count,Volume BTC
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 02:00:00,0.224018,0.561453,0.534654,0.534654,0.693206,0.693206,0.585239,0.585239,0.670205,0.670205,...,0.053190,0.053190,0.073059,0.073059,0.073059,0.129999,0.129999,0.129999,0.072129,0.053893
2021-01-01 03:00:00,0.225666,0.585239,0.561453,0.561453,0.534654,0.534654,0.670205,0.670205,0.732950,0.732950,...,0.069336,0.069336,0.129999,0.129999,0.129999,0.072129,0.072129,0.072129,0.053190,0.033032
2021-01-01 04:00:00,0.224523,0.670205,0.585239,0.585239,0.561453,0.561453,0.732950,0.732950,0.562987,0.562987,...,0.052302,0.052302,0.072129,0.072129,0.072129,0.053190,0.053190,0.053190,0.069336,0.046068
2021-01-01 05:00:00,0.223868,0.732950,0.670205,0.670205,0.585239,0.585239,0.562987,0.562987,0.697251,0.697251,...,0.058058,0.058058,0.053190,0.053190,0.053190,0.069336,0.069336,0.069336,0.052302,0.033227
2021-01-01 06:00:00,0.223619,0.562987,0.732950,0.732950,0.670205,0.670205,0.697251,0.697251,0.670408,0.670408,...,0.066514,0.066514,0.069336,0.069336,0.069336,0.052302,0.052302,0.052302,0.058058,0.032114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-30 18:00:00,0.253141,0.587794,0.591526,0.591526,0.519697,0.519697,0.617700,0.617700,0.470314,0.470314,...,0.049669,0.049669,0.052216,0.052216,0.052216,0.043628,0.043628,0.043628,0.030513,0.018008
2022-05-30 19:00:00,0.252517,0.617700,0.587794,0.587794,0.591526,0.591526,0.470314,0.470314,0.458503,0.458503,...,0.158561,0.158561,0.043628,0.043628,0.043628,0.030513,0.030513,0.030513,0.049669,0.089353
2022-05-30 20:00:00,0.264993,0.470314,0.617700,0.617700,0.587794,0.587794,0.458503,0.458503,0.487817,0.487817,...,0.231511,0.231511,0.030513,0.030513,0.030513,0.049669,0.049669,0.049669,0.158561,0.304045


In [28]:
sorted_df.keys

<bound method NDFrame.keys of                         Close  sentiment_polarization  Close_1hr  \
datetime                                                           
2021-01-01 02:00:00  0.224018                0.561453   0.534654   
2021-01-01 03:00:00  0.225666                0.585239   0.561453   
2021-01-01 04:00:00  0.224523                0.670205   0.585239   
2021-01-01 05:00:00  0.223868                0.732950   0.670205   
2021-01-01 06:00:00  0.223619                0.562987   0.732950   
...                       ...                     ...        ...   
2022-05-30 18:00:00  0.253141                0.587794   0.591526   
2022-05-30 19:00:00  0.252517                0.617700   0.587794   
2022-05-30 20:00:00  0.264993                0.470314   0.617700   
2022-05-30 21:00:00  0.275437                0.458503   0.470314   
2022-05-30 22:00:00  0.274659                0.487817   0.458503   

                     sentiment_lag_1hr  Close_2hr  sentiment_lag_2hr  \
datetime     

In [41]:
sorted_df.to_csv('final_data_LSTM.csv')

## Spliting the data into training data and test data

In [8]:
test_split=round(len(sorted_df)*0.30)
test_split

3692

In [9]:
df_for_training=sorted_df[:-3692]
df_for_testing=sorted_df[-3692:]
print(df_for_training.shape)
print(df_for_testing.shape)

(8615, 25)
(3692, 25)


In [31]:
df_for_training

,Close,sentiment_polarization,Close_1hr,sentiment_lag_1hr,Close_2hr,sentiment_lag_2hr,Close_lag_back_1hr,sentiment_lag_back_1hr,sentiment_lag_back_2hr,Close_lag_back_2hr,...,Volume USDT_back_1hr,trade_count_back_1hr,trade_count_2hr,Volume USDT_2hr,Volume BTC_2hr,Volume USDT_1hr,trade_count_1hr,Volume BTC_1hr,trade_count,Volume BTC
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 02:00:00,0.224018,0.561453,0.534654,0.534654,0.693206,0.693206,0.585239,0.585239,0.670205,0.670205,...,0.053190,0.053190,0.073059,0.073059,0.073059,0.129999,0.129999,0.129999,0.072129,0.053893
2021-01-01 03:00:00,0.225666,0.585239,0.561453,0.561453,0.534654,0.534654,0.670205,0.670205,0.732950,0.732950,...,0.069336,0.069336,0.129999,0.129999,0.129999,0.072129,0.072129,0.072129,0.053190,0.033032
2021-01-01 04:00:00,0.224523,0.670205,0.585239,0.585239,0.561453,0.561453,0.732950,0.732950,0.562987,0.562987,...,0.052302,0.052302,0.072129,0.072129,0.072129,0.053190,0.053190,0.053190,0.069336,0.046068
2021-01-01 05:00:00,0.223868,0.732950,0.670205,0.670205,0.585239,0.585239,0.562987,0.562987,0.697251,0.697251,...,0.058058,0.058058,0.053190,0.053190,0.053190,0.069336,0.069336,0.069336,0.052302,0.033227
2021-01-01 06:00:00,0.223619,0.562987,0.732950,0.732950,0.670205,0.670205,0.697251,0.697251,0.670408,0.670408,...,0.066514,0.066514,0.069336,0.069336,0.069336,0.052302,0.052302,0.052302,0.058058,0.032114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 03:00:00,0.653226,0.488066,0.516584,0.516584,0.438743,0.438743,0.488359,0.488359,0.417951,0.417951,...,0.042511,0.042511,0.036271,0.036271,0.036271,0.037597,0.037597,0.037597,0.041486,0.025082
2021-12-27 04:00:00,0.653078,0.488359,0.488066,0.488066,0.516584,0.516584,0.417951,0.417951,0.434106,0.434106,...,0.031349,0.031349,0.037597,0.037597,0.037597,0.041486,0.041486,0.041486,0.042511,0.024409
2021-12-27 05:00:00,0.650157,0.417951,0.488359,0.488359,0.488066,0.488066,0.434106,0.434106,0.560692,0.560692,...,0.035480,0.035480,0.041486,0.041486,0.041486,0.042511,0.042511,0.042511,0.031349,0.016887


In [10]:
X_train = df_for_training.drop('Close', axis=1)

X_test = df_for_testing.drop('Close', axis=1)

y_train = df_for_training[['Close']]

y_test = df_for_testing[['Close']]

Linear Regression

In [35]:

# Define a pipeline with preprocessing steps and Linear Regression model
steps = [
    ('poly', PolynomialFeatures()),  # Polynomial feature transformation
    ('model', LinearRegression())  # Linear Regression model
]

pipeline = Pipeline(steps)

# Define hyperparameters to tune
parameters = {
    'poly__degree': [1, 2, 3, 4],  # Degree of polynomial features
    'model__fit_intercept': [True, False],  # Whether to fit intercept
}

# Perform grid search cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)



print('Best parameters:', grid_search.best_params_)
print('Best R-squared on training set:', grid_search.best_score_)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

Best parameters: {'model__fit_intercept': True, 'poly__degree': 2}
Best R-squared on training set: -0.016169666259423107
R-squared on test set: 0.43397319594910966
Mean Absolute Error on test set: 0.05642895320164709
Mean Squared Error on test set: 0.005573676352646528
Root Mean Squared Error on test set: 0.07465705829087112


Testing the number of variables with the performance of linear regresion model as baseline

In [33]:
# Assuming X_train, X_test, y_train, y_test, and df_for_training are defined

for idx, column_name in enumerate(X_train.columns, 1):
    model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression(fit_intercept=False))
    X_train_subset = X_train.iloc[:, :idx]
    X_test_subset = X_test.iloc[:, :idx]
    model.fit(X_train_subset, y_train)

    y_pred = model.predict(X_test_subset)
    test_r2 = r2_score(y_test, y_pred)
    mae_score = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    
    print(f'Model performance with {idx} variables ({column_name})')
    print('R-squared on test set:', test_r2)
    print('Mean Squared Error on test set:', mse)
    print('--------------------------------------------')

Model performance with 1 variables (sentiment_polarization)
R-squared on test set: -0.7211166348930924
Mean Squared Error on test set: 0.016947867167060367
--------------------------------------------
Model performance with 2 variables (Close_1hr)
R-squared on test set: -0.6155921509544895
Mean Squared Error on test set: 0.01590876563239004
--------------------------------------------
Model performance with 3 variables (sentiment_lag_1hr)
R-squared on test set: -0.6155921509544957
Mean Squared Error on test set: 0.015908765632390104
--------------------------------------------
Model performance with 4 variables (Close_2hr)
R-squared on test set: -0.5683911118152001
Mean Squared Error on test set: 0.015443976131631088
--------------------------------------------
Model performance with 5 variables (sentiment_lag_2hr)
R-squared on test set: -0.5683911118151994
Mean Squared Error on test set: 0.015443976131631083
--------------------------------------------
Model performance with 6 variabl

Decision tree

Testing the number of variables with the performance of descision tree model as baseline

In [36]:
for idx, column_name in enumerate(X_train.columns, 1):
    model = DecisionTreeRegressor(random_state=42)
    X_train_subset = X_train.iloc[:, :idx]
    X_test_subset = X_test.iloc[:, :idx]
    model.fit(X_train_subset, y_train)

    y_pred = model.predict(X_test_subset)
    test_r2 = r2_score(y_test, y_pred)
    mae_score = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    
    print(f'Model performance with {idx} variables ({column_name})')
    print('R-squared on test set:', test_r2)
    print('Mean Squared Error on test set:', mse)
    print('--------------------------------------------')

Model performance with 1 variables (sentiment_polarization)
R-squared on test set: -3.7501743037573183
Mean Squared Error on test set: 0.04677505375773857
--------------------------------------------
Model performance with 2 variables (Close_1hr)
R-squared on test set: -3.6478964747589115
Mean Squared Error on test set: 0.04576792209399285
--------------------------------------------
Model performance with 3 variables (sentiment_lag_1hr)
R-squared on test set: -3.645373799529321
Mean Squared Error on test set: 0.045743081264597585
--------------------------------------------
Model performance with 4 variables (Close_2hr)
R-squared on test set: -3.814312798988299
Mean Squared Error on test set: 0.04740662670022965
--------------------------------------------
Model performance with 5 variables (sentiment_lag_2hr)
R-squared on test set: -3.784124778239894
Mean Squared Error on test set: 0.04710936470455311
--------------------------------------------
Model performance with 6 variables (Cl

In [37]:
# Define hyperparameters to tune
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create Decision Tree regressor
tree_reg = DecisionTreeRegressor(random_state=42)

# Perform grid search cross-validation
grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)


print('Best parameters:', grid_search.best_params_)
print('Best R-squared on training set:', grid_search.best_score_)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

Best parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best R-squared on training set: -0.0033805339157555077
R-squared on test set: 0.5528336959441994
Mean Absolute Error on test set: 0.03991579782439571
Mean Squared Error on test set: 0.004403254822526178
Root Mean Squared Error on test set: 0.06635702541951513


Random Forest

In [12]:
# Convert target variables to 1D arrays
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Define hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create Random Forest regressor
rf_reg = RandomForestRegressor(random_state=42)

# Perform grid search cross-validation
grid_search = GridSearchCV(rf_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)


print('Best parameters:', grid_search.best_params_)
print('Best R-squared on training set:', grid_search.best_score_)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best R-squared on training set: -0.002388673696023043
R-squared on test set: 0.8156629319167173
Mean Absolute Error on test set: 0.021570355753062372
Mean Squared Error on test set: 0.001815170500652847
Root Mean Squared Error on test set: 0.042604817810346834


XGBoost

In [11]:
regressor=xgb.XGBRegressor(tree_method="gpu_hist")

# set up our search grid
param_grid = {"max_depth":    [None, 5, 10, 15],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error').fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

The best hyperparameters are  {'learning_rate': 0.015, 'max_depth': None, 'n_estimators': 700}


In [12]:
regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],
                           )

regressor.fit(X_train, y_train)

predictions = regressor.predict(X_test)

test_r2 = r2_score(y_test, predictions)
mae_score = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = math.sqrt(mse)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

R-squared on test set: 0.7965230640262182
Mean Absolute Error on test set: 0.025767671827073235
Mean Squared Error on test set: 0.0020036411318854665
Root Mean Squared Error on test set: 0.044762050130500795
